<a href="https://colab.research.google.com/github/hugosousa111/ph2_dataset_lesion_features/blob/main/PH2_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Acessa o Drive
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Bibliotecas
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import random
random.seed(0)

In [ ]:
# Pegando a base
df = pd.read_excel('/content/drive/My Drive/PH2Dataset/PH2_dataset.xlsx')
df.columns = ["Image Name","Histological Diagnosis","Common Nevus","Atypical Nevus", "Melanoma", "Asymmetry(0/1/2)", "Pigment Network(AT/T)", "Dots/Globules(A/AT/T)", "Streaks\n(A/P)","Regression Areas(A/P)", "Blue-Whitish Veil(A/P)", "White", "Red", "Light-Brown", "Dark-Brown", "Blue-Gray", "Black"]

# Tratando a legenda
legenda = df.iloc[1:8,]
legenda = legenda[['Pigment Network(AT/T)','Dots/Globules(A/AT/T)']]
legenda.columns = ['Sigla', 'Significado']
legenda.reset_index

# Apenas os dados
df = df.iloc[12:,] #apagando legenda
df = df.reset_index(drop=True)

In [ ]:
#df
#legenda

In [ ]:
# Função que pega o rótulo de uma amostra dada o seu nome
def get_rotulo(img, df):
    if df["Common Nevus"][df["Image Name"] == img].values == 'X':
        return 0
    elif df["Atypical Nevus"][df["Image Name"] == img].values  == 'X':
        return 1
    else:
        return 2

# Coloco os rótulos em 0, 1 e 2
Images_Name_column = df['Image Name']
rotulos = np.zeros(200)
cont = 0
for i in Images_Name_column: 
    rotulos[cont] = get_rotulo(i, df)
    cont +=1

# Coloca esse vetor de rotulos em uma das colunas
df['Common Nevus'] = rotulos.astype(int)
df=df.rename(columns = {'Common Nevus':'Label'})
df

# Apaga as outras colunas
df = df.drop(columns=['Atypical Nevus','Melanoma'])

In [ ]:
# Procurando em cada coluna se tem NaN
ver = 0
if ver == 1: 
    print('Image Name')
    print(df['Image Name'].isnull().sum())
    print('Histological Diagnosis')
    print(df['Histological Diagnosis'].isnull().sum())
    print("Asymmetry(0/1/2)")
    print(df["Asymmetry(0/1/2)"].isnull().sum())
    print("Pigment Network(AT/T)")
    print(df["Pigment Network(AT/T)"].isnull().sum())
    print("Dots/Globules(A/AT/T)")
    print(df["Dots/Globules(A/AT/T)"].isnull().sum())
    print("Streaks\n(A/P)")
    print(df["Streaks\n(A/P)"].isnull().sum())
    print("Regression Areas(A/P)")
    print(df["Regression Areas(A/P)"].isnull().sum())
    print("Blue-Whitish Veil(A/P)")
    print(df["Blue-Whitish Veil(A/P)"].isnull().sum())
    print("White")
    print(df["White"].isnull().sum())
    print("Red")
    print(df["Red"].isnull().sum())
    print("Light-Brown")
    print(df["Light-Brown"].isnull().sum())
    print("Dark-Brown")
    print(df["Dark-Brown"].isnull().sum())
    print("Blue-Gray")
    print(df["Blue-Gray"].isnull().sum())
    print("Black")
    print(df["Black"].isnull().sum())

In [ ]:
# Tratando valores NaN nas colunas de cores
col = ["White", "Red", "Light-Brown", "Dark-Brown", "Blue-Gray", "Black"]
for c in col: 
    vet = np.zeros(200)
    cont = 0
    for el in df[c]:
        if el == 'X':
            vet[cont] = 1
        cont += 1
    df[c] = vet.astype(int)

In [ ]:
# Fazendo o LabelEncoder nos atributos string
le = LabelEncoder()
df['Pigment Network(AT/T)'] = le.fit_transform(df['Pigment Network(AT/T)'])
df['Dots/Globules(A/AT/T)'] = le.fit_transform(df['Dots/Globules(A/AT/T)'])
df['Streaks\n(A/P)'] = le.fit_transform(df['Streaks\n(A/P)'])
df['Regression Areas(A/P)'] = le.fit_transform(df['Regression Areas(A/P)'])
df['Blue-Whitish Veil(A/P)'] = le.fit_transform(df['Blue-Whitish Veil(A/P)'])

# Fazendo o OneHotEncoder
x = df[['Pigment Network(AT/T)','Dots/Globules(A/AT/T)','Streaks\n(A/P)','Regression Areas(A/P)','Blue-Whitish Veil(A/P)']]
y = OneHotEncoder().fit_transform(x).toarray()

# Contatenando com a base do onehotenc
col = ['Pigment Network(AT/T)_1', 'Pigment Network(AT/T)_2','Dots/Globules(A/AT/T)_1','Dots/Globules(A/AT/T)_2','Dots/Globules(A/AT/T)_3','Streaks\n(A/P)_1','Streaks\n(A/P)_2','Regression Areas(A/P)_1','Regression Areas(A/P)_2','Blue-Whitish Veil(A/P)_1','Blue-Whitish Veil(A/P)_2']
df2 = pd.DataFrame(y, columns=col).astype(int)

df_ = pd.concat([df, df2], axis=1)

# Apagando colunas antigas
df_ = df_.drop(columns=['Pigment Network(AT/T)','Dots/Globules(A/AT/T)','Streaks\n(A/P)','Regression Areas(A/P)','Blue-Whitish Veil(A/P)'])

In [ ]:
# Dividindo em atributos e labels
com_one_hot = 1
if com_one_hot == 1:
    df_atributos = df_.iloc[:, 3:21].values
    df_label = df_.iloc[:, 2].values
else:
    df_atributos = df.iloc[:, 3:15].values
    df_label = df.iloc[:, 2].values

In [ ]:
# Padronização # TA ERRADO, tem q ser feito depois
com_padronizacao = 0
if com_padronizacao == 1:
    scl = StandardScaler()
    df_atributos = scl.fit_transform(df_atributos)

In [ ]:
# Dividindo em treino e teste
df_atributos_train, df_atributos_test, df_label_train, df_label_test = train_test_split(df_atributos, df_label, test_size=0.25)

In [ ]:
#classificador = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
#classificador = DecisionTreeClassifier(criterion='entropy')
#classificador = GaussianNB()
#classificador = RandomForestClassifier(n_estimators = 5,criterion='entropy')
#classificador = LogisticRegression()
classificador = SVC(kernel = 'rbf')

classificador.fit(df_atributos_train, df_label_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
previsoes = classificador.predict(df_atributos_test)

In [ ]:
accuracy_score(df_label_test, previsoes)

0.85

In [ ]:
confusion_matrix(df_label_test, previsoes)

array([[16,  0,  0],
       [ 1, 12,  3],
       [ 0,  2,  6]])

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state = 0, shuffle=True)

ac = np.zeros(5)
cont = 0
for train_index, test_index in kf.split(df_atributos):
    #print("\n\nTRAIN:", train_index, "\nTEST:", test_index)
    df_atributos_train, df_atributos_test = df_atributos[train_index], df_atributos[test_index]
    df_label_train, df_label_test = df_label[train_index], df_label[test_index]
    
    classificador.fit(df_atributos_train, df_label_train)
    previsoes = classificador.predict(df_atributos_test)
    
    #print("Confusion Matrix")
    #print(confusion_matrix(df_label_test, previsoes))
    #print()

    ac[cont] = accuracy_score(df_label_test, previsoes)
    cont += 1

print(ac)
print("Acuracia: %0.2f" % (ac.mean()))
print("STD: %0.2f" % (ac.std()))

[0.875 0.95  0.875 0.9   0.85 ]
Acuracia: 0.89
STD: 0.03


In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state = 0, shuffle=True)

ac = np.zeros(5)
cont = 0
for train_index, test_index in skf.split(df_atributos, df_label):
    #print("\n\nTRAIN:", train_index, "\nTEST:", test_index)
    df_atributos_train, df_atributos_test = df_atributos[train_index], df_atributos[test_index]
    df_label_train, df_label_test = df_label[train_index], df_label[test_index]
    
    classificador.fit(df_atributos_train, df_label_train)
    previsoes = classificador.predict(df_atributos_test)
    
    #print("Confusion Matrix")
    #print(confusion_matrix(df_label_test, previsoes))
    #print()

    ac[cont] = accuracy_score(df_label_test, previsoes)
    cont += 1

print(ac)
print("Acuracia: %0.2f" % (ac.mean()))
print("STD: %0.2f" % (ac.std()))

[0.85  0.925 0.825 0.9   0.85 ]
Acuracia: 0.87
STD: 0.04
